In [1]:
import requests
from bs4 import BeautifulSoup
flipkart_data=[]
for i in range(0,40):
    # Looping Through 40 Pages as at the Link remains the same only the Page number gets changes
    link = "https://www.flipkart.com/search?q=mobiles&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=off&as=off&page=%d" % (i+1)
    page=requests.get(link)
    dataflipkart=BeautifulSoup(page.text,"html.parser")
    # Getting The common Div Class which stores the Mobile Information
    for a in dataflipkart.findAll('div', attrs={'class':'_3pLy-c row'}):
        flipkart_temp={}
        # Getting the Mobile Name
        name=a.find('div', attrs={'class':'_4rR01T'})
        flipkart_temp['MobileName']=name.text
        #Extracting the Company Name from Mobile
        flipkart_temp['CompanyName'] = flipkart_temp['MobileName'].split()[0]
        # Getting the Number of Stars that Mobile Has
        try:
            flipkart_temp['Stars']=a.find('div', attrs={'class':'_3LWZlK'}).text.strip()
        except AttributeError:
            flipkart_temp['Stars'] = 0
        #Getting the Ratings the Mobile has Got
        try:
            ratings=a.find('span',attrs={'class':'_2_R_DZ'}).text.strip()
            flipkart_temp['Ratings']=ratings.split()[0]
        except AttributeError:
            flipkart_temp['Ratings'] = 0
        #Getting the Reviews the Mobile has Got
        try:
            ratings=a.find('span',attrs={'class':'_2_R_DZ'}).text.strip()
            flipkart_temp['Reviews']=ratings.split()[3]
        except AttributeError:
            flipkart_temp['Reviews']=0
        #Getting the Discounted Price
        discountedprice=a.find('div',attrs={'class':'_30jeq3 _1_WHN1'}).text.strip()
        flipkart_temp['Discount_Price']=discountedprice.replace('₹','')
        #Getting the Actual Price
        try:
            flipkart_temp['Actual_Price'] = a.find('div',{'class':'_3I9_wc _27UcVY'}).text.replace('₹','').strip()
        except AttributeError:
            flipkart_temp['Actual_Price'] = flipkart_temp['Discount_Price']
        #Getting The Discount Provided by the Mobile Company
        try:
            flipkart_temp['Discount'] = a.find('div',{'class':'_3Ay6Sb'}).text.replace('% off','').strip()
        except AttributeError:
            flipkart_temp['Discount']=0
        #Getting how much exchange Max offered if a mobile is exchanged
        for d in a.findAll('div',attrs={'class':'_18hQoS'}):
            if '₹' in d.text:
                flipkart_temp['Exchange_Off']=d.text.replace('₹','')
        #Looping through the specs of the Mobile        
        for d in a.findAll('li',attrs={'class':'rgWa7D'}):
            #Getting the RAM
            if 'GB RAM' in d.text:
                flipkart_temp['RAM_IN_GB']=d.text.split('|')[0].split()[0]
            #Getting the Internal Storage
            if 'GB ROM' in d.text:
                if 'GB RAM' in d.text:
                    flipkart_temp['Internal_Storage']=d.text.split('|')[1].split()[0].replace('GB','').strip()
                else:
                    flipkart_temp['Internal_Storage']=d.text.split()[0].strip()
            #Getting the till how much can the Storage be expanded
            if 'Expandable' in d.text:
                flipkart_temp['Expandable_Upto']=d.text.split('Upto')[1].strip()
            if 'inch' in d.text:
                flipkart_temp['Display_In_Inch']=d.text.split('(')[1].split()[0].strip()
            #Getting the Back Camera Details
            if 'Camera' in d.text:
                if 'MP' in d.text:
                    if d.text[0].isdigit()== True:
                        flipkart_temp['Main_Camera']=d.text.split('|')[0].split()[0].replace('MP','').strip()
            #Getting the Battery Details
            if 'mAh' in d.text:
                flipkart_temp['Battery']=d.text.split()[0].strip()
            #Getting the Processor Details
            if 'Processor' in d.text:
                flipkart_temp['Processor']=d.text.replace('Processor','').strip()
        #Appending the Details into a list
        flipkart_data.append(flipkart_temp)
print("Data Extraction Complete")

Data Extraction Complete


In [2]:
import pandas as pd
#Putting the List into a DataFrame
df_flipkart = pd.DataFrame(flipkart_data)
#Extracting the Dataframe to a CSV File
df_flipkart.to_csv('Flipkart_Data.csv')